In [1]:
import math
from logging import debug

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil
from scipy.stats import linregress
import libraries as lib

In [2]:
directory = "../../../OneDrive - HvA/Logfiles/SOLAR/2024/20240707_20240715_terugvaart/20240713_Huy-Limburg/0026 - 20240713T080342 14h37m16s 145.044km Durnal - Heel"
result1 = lib.DataUitzoekenGui(directory)

SelectMultiple(description='Bestanden:', options=('0_Bus.csv', '2_MPPT_0_05_05.csv', '3_MPPT_1_05_07.csv', '4_…

Button(button_style='success', description='Bestanden Laden', icon='check', style=ButtonStyle(), tooltip='Klik…

Label(value='')

In [3]:
# Na het laden van de bestanden en het klikken op de knop
# Controleer of 'merged_df' in result zit
if 'merged_df' in result1:
    merged_df = result1['merged_df'].copy()
    # Gebruik merged_df zoals je wilt
else:
    print("De data is nog niet geladen.")
merged_df

,1_Datalogger port,"1_Dataloggertijd, in s",1_Cycle count,1_Tijd,1_Latitude,1_Longitude,1_Snelheid over de grond in km/h,1_Richting van de snelheid,1_Datum,1_Magnetische directie,...,2_CRC errors detected (0),2_Unknown packets (0),2_Number of times more or fewer fields than expected in a packet (0),2_gps long,2_gps lat,2_gps direction,2_gps speed,2_gps time,2_Datalogger port,2_Format header (>20|02)
Indextijd,,,,,,,,,,,,,,,,,,,,,
0 days 00:02:19,9.0,139.0348,143.630,60343.0,5020.2014,459.8611,0.0,9.7,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:02:20,9.0,140.0466,144.642,60344.0,5020.2014,459.8611,0.0,9.7,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:02:21,9.0,141.0361,145.631,60345.0,5020.2014,459.8611,0.0,9.7,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:02:22,9.0,142.0393,146.635,60346.0,5020.2014,459.8611,0.0,9.7,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:02:23,9.0,143.0548,147.651,60347.0,5020.2014,459.8611,0.0,9.7,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 14:39:30,9.0,52770.1838,52782.637,204054.0,5110.7016,552.5635,0.0,286.3,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 14:39:31,9.0,52771.1616,52783.613,204055.0,5110.7016,552.5635,0.0,286.3,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 14:39:32,9.0,52772.2290,52784.684,204056.0,5110.7016,552.5635,0.0,286.3,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#filter de eerste 100 waarders erut
merged_df1 = merged_df.iloc[17500:-1200]
lib.plot_trajectory_map(merged_df1['1_Latitude'].dropna(), merged_df1['1_Longitude'].dropna(), merged_df1['1_Snelheid over de grond in km/h'].dropna())

In [27]:
# Only use the data where the speed isn't nan
merged_df1 = merged_df1[merged_df1['1_Snelheid over de grond in km/h'].notna()]
merged_df1

,1_Datalogger port,"1_Dataloggertijd, in s",1_Cycle count,1_Tijd,1_Latitude,1_Longitude,1_Snelheid over de grond in km/h,1_Richting van de snelheid,1_Datum,1_Magnetische directie,...,2_CRC errors detected (0),2_Unknown packets (0),2_Number of times more or fewer fields than expected in a packet (0),2_gps long,2_gps lat,2_gps direction,2_gps speed,2_gps time,2_Datalogger port,2_Format header (>20|02)
Indextijd,,,,,,,,,,,,,,,,,,,,,
0 days 04:53:59,9.0,17639.2439,17646.455,105523.0,5035.4557,527.4641,0.0,92.5,130724.0,NaN,...,0.0,0.0,527.4641,5035.4557,92.5,0.0,105522.0,NaN,B,20|02
0 days 04:54:00,9.0,17640.2068,17647.418,105524.0,5035.4557,527.4641,0.0,92.5,130724.0,NaN,...,0.0,0.0,527.4641,5035.4557,92.5,0.0,105523.0,NaN,B,20|02
0 days 04:54:01,9.0,17641.2219,17648.434,105525.0,5035.4557,527.4641,0.0,92.5,130724.0,NaN,...,0.0,0.0,527.4641,5035.4557,92.5,0.0,105524.0,NaN,B,20|02
0 days 04:54:02,9.0,17642.2337,17649.445,105526.0,5035.4557,527.4641,0.0,92.5,130724.0,NaN,...,0.0,0.0,527.4641,5035.4557,92.5,0.0,105525.1,NaN,B,20|02
0 days 04:54:03,9.0,17643.2083,17650.420,105527.0,5035.4557,527.4641,0.0,92.5,130724.0,NaN,...,0.0,0.0,527.4641,5035.4557,92.5,0.0,105526.2,NaN,B,20|02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 14:19:30,9.0,51570.2356,51582.508,202054.0,5110.6936,552.6095,0.0,268.4,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 14:19:31,9.0,51571.2207,51583.492,202055.0,5110.6936,552.6095,0.0,268.4,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 14:19:32,9.0,51572.2591,51584.531,202056.0,5110.6936,552.6095,0.0,268.4,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Filter the current data into 3 sections, one when the column '1_Tijd' is between 110000 and 133000, one between 133000 and 200000 and one between 2000 and the end
merged_df1_1 = merged_df1[(merged_df1['1_Tijd'] >= 110000) & (merged_df1['1_Tijd'] <= 133000)]
merged_df1_2 = merged_df1[(merged_df1['1_Tijd'] >= 133000) & (merged_df1['1_Tijd'] <= 200000)]
merged_df1_3 = merged_df1[(merged_df1['1_Tijd'] >= 200000)]

#lib.plot_trajectory_map(merged_df1_2['1_Latitude'].dropna(), merged_df1_2['1_Longitude'].dropna(), merged_df1_2['1_Snelheid over de grond in km/h'].dropna())
merged_df1_3

,1_Datalogger port,"1_Dataloggertijd, in s",1_Cycle count,1_Tijd,1_Latitude,1_Longitude,1_Snelheid over de grond in km/h,1_Richting van de snelheid,1_Datum,1_Magnetische directie,...,2_CRC errors detected (0),2_Unknown packets (0),2_Number of times more or fewer fields than expected in a packet (0),2_gps long,2_gps lat,2_gps direction,2_gps speed,2_gps time,2_Datalogger port,2_Format header (>20|02)
Indextijd,,,,,,,,,,,,,,,,,,,,,
0 days 13:58:36,9.0,50316.1849,50328.273,200000.0,5110.0001,553.4569,7.0,344.3,130724.0,NaN,...,0.0,0.0,553.457350,5109.999110,345.110000,7.180000,196363.100000,NaN,B,20|02
0 days 13:58:37,9.0,50317.2482,50329.336,200001.0,5110.0012,553.4563,7.2,343.4,130724.0,NaN,...,0.0,0.0,553.456900,5110.000100,344.300000,7.000000,200000.000000,NaN,B,20|02
0 days 13:58:38,9.0,50318.2186,50330.305,200002.0,5110.0022,553.4560,7.0,342.3,130724.0,NaN,...,0.0,0.0,553.456233,5110.001422,343.155556,7.155556,200001.222222,NaN,B,20|02
0 days 13:58:39,9.0,50319.1809,50331.266,200003.0,5110.0032,553.4555,7.0,341.3,130724.0,NaN,...,0.0,0.0,553.456000,5110.002200,342.300000,7.000000,200002.000000,NaN,B,20|02
0 days 13:58:40,9.0,50320.1794,50332.266,200004.0,5110.0044,553.4550,7.0,339.6,130724.0,NaN,...,0.0,0.0,553.455500,5110.003200,341.300000,7.000000,200003.000000,NaN,B,20|02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 14:19:30,9.0,51570.2356,51582.508,202054.0,5110.6936,552.6095,0.0,268.4,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 14:19:31,9.0,51571.2207,51583.492,202055.0,5110.6936,552.6095,0.0,268.4,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 14:19:32,9.0,51572.2591,51584.531,202056.0,5110.6936,552.6095,0.0,268.4,130724.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
